# Data Wrangling

This first capstone is intended to forecast the consumption of red meat in the United States over the next 10 years. We will look for patterns in potential social, economic, and environmental indicators that could be predictors of consumption. 

In this notebook, we will inspect and clean our datasets for this project. The way that FAO stores their data, they have two separate sheets for data before 2013 and after. Since we have the same number of columns in each, our first step will be to combine these into one.

In [1]:
import pandas as pd
fao08 = pd.read_csv('faostat_08.csv')
fao14 = pd.read_csv('faostat_14.csv')

#First we concat our dataframes together, since they have the same column structure. Then we reset the index and drop the old one.
fao_all = pd.concat([fao08,fao14])
fao_all.reset_index(inplace=True)
fao_all.drop(labels='index', axis=1, inplace=True)
fao_all.head()


,Domain,Area,Element,Item,Year,Unit,Value
0,"Food Balances (-2013, old methodology and popu...",United States of America,Production,Bovine Meat,2008,1000 tonnes,12163.0
1,"Food Balances (-2013, old methodology and popu...",United States of America,Production,Bovine Meat,2009,1000 tonnes,11891.0
2,"Food Balances (-2013, old methodology and popu...",United States of America,Production,Bovine Meat,2010,1000 tonnes,12046.0
3,"Food Balances (-2013, old methodology and popu...",United States of America,Production,Bovine Meat,2011,1000 tonnes,11921.0
4,"Food Balances (-2013, old methodology and popu...",United States of America,Production,Bovine Meat,2012,1000 tonnes,11811.0


In [2]:
fao_all.dtypes

Domain      object
Area        object
Element     object
Item        object
Year         int64
Unit        object
Value      float64
dtype: object

In [3]:
fao_all.Element.unique()

array(['Production', 'Import Quantity', 'Stock Variation',
       'Export Quantity', 'Losses', 'Processing', 'Other uses (non-food)',
       'Tourist consumption', 'Residuals'], dtype=object)

In [4]:
fao_all.Item.unique()

array(['Bovine Meat', 'Mutton & Goat Meat', 'Pigmeat', 'Poultry Meat',
       'Meat, Other'], dtype=object)

In [5]:
fao_all.Unit.unique()

array(['1000 tonnes', nan], dtype=object)

In [6]:
fao_all['Domain'].unique()

array(['Food Balances (-2013, old methodology and population)',
       'Food Balances (2014-)'], dtype=object)

In [7]:
fao_all[(fao_all.Item=='Pigmeat') & (fao_all.Year==2008)]

,Domain,Area,Element,Item,Year,Unit,Value
108,"Food Balances (-2013, old methodology and popu...",United States of America,Production,Pigmeat,2008,1000 tonnes,10599.0
114,"Food Balances (-2013, old methodology and popu...",United States of America,Import Quantity,Pigmeat,2008,1000 tonnes,454.0
120,"Food Balances (-2013, old methodology and popu...",United States of America,Stock Variation,Pigmeat,2008,1000 tonnes,-19.0
126,"Food Balances (-2013, old methodology and popu...",United States of America,Export Quantity,Pigmeat,2008,1000 tonnes,2129.0
132,"Food Balances (-2013, old methodology and popu...",United States of America,Losses,Pigmeat,2008,NaN,NaN
138,"Food Balances (-2013, old methodology and popu...",United States of America,Processing,Pigmeat,2008,1000 tonnes,0.0
144,"Food Balances (-2013, old methodology and popu...",United States of America,Other uses (non-food),Pigmeat,2008,1000 tonnes,15.0
150,"Food Balances (-2013, old methodology and popu...",United States of America,Tourist consumption,Pigmeat,2008,NaN,NaN
156,"Food Balances (-2013, old methodology and popu...",United States of America,Residuals,Pigmeat,2008,NaN,NaN


It looks like the "Domain" column is not relevant to our analysis - it only speaks to what methodology the balance sheet used - so we'll drop it

In [8]:
fao_all.drop(labels='Domain', axis=1, inplace=True)

## Null Values
Next we'll check for null values. We can see from the output above, that the numeric data we care most about is the Value column. We'll want to check if and when values are null and if that corresponds to null columns.

In [9]:
null_vals = fao_all[fao_all.Value.isna()]
fao_all.drop(labels = null_vals.index, inplace=True)

## Understanding "Elements"
We don't have an precise measure for food consumption yet. We have a variety of "Elements" whose values can be taken together to measure how much meat was consumed. We don't have a sense of what the range of values here is and what makes sense to subtract and add together. Let's see what the mean value for each element can tell us.

In [10]:
fao_means = fao_all.groupby('Element')['Value'].mean()
fao_means

Element
Export Quantity          1464.254545
Import Quantity           407.090909
Other uses (non-food)     159.393939
Processing                  7.121212
Production               8687.309091
Residuals                 -19.640000
Stock Variation             9.200000
Name: Value, dtype: float64

A few interesting things stand out: Production is by far the highest. That makes sense because the United States is a big producer of meat. That goes hand in hand with being a strong exporter of meat, which is the next highest mean. The rest of our values are a lot lower by comparison. Through some googling we have learned that Processing, Other uses, Stock variation, and Residuals all *take away* from the overall production value. These are associated with food being lost before it reaches a consumer, so we'll want to subtract them from the overall production value. The one exception is Residuals, which have a negative value in this case. We'll want to add those operationally to reflect the amount lost.

In [11]:
#confirm residuals is always negative
fao_all[fao_all.Element=='Residuals']['Value'].max()

0.0

In [12]:
#Multiple all element values by -1 where they should be subtracted
neg_elems = ['Export Quantity', 'Processing','Stock Variation','Other uses (non-food)']
for neg in neg_elems:
    fao_all.loc[fao_all.Element==neg, ['Value']] = fao_all.loc[fao_all.Element==neg, ['Value']].apply(lambda x:x*-1)

In [15]:
#If we want to see averages for each element per year
avg_elems = fao_all.set_index('Year').groupby(['Element']).mean()
avg_elems[:]

,Value
Element,
Export Quantity,-1464.254545
Import Quantity,407.090909
Other uses (non-food),-159.393939
Processing,-7.121212
Production,8687.309091
Residuals,-19.640000
Stock Variation,-9.200000


In [30]:
#If we want to see average values for each item per year
all_avg = fao_all.set_index('Item').groupby(['Year']).mean()
all_avg

,Value
Year,
2008,1448.538462
2009,1415.192308
2010,1427.192308
2011,1386.500000
2012,1401.192308
2013,1414.076923
2014,1186.677419
2015,1214.709677
2016,1243.064516


## Conumption value
Now that we have identified which of our values are negative, we can sum the entire value column for each food type to get an overall consumption number in a new dataframe.

In [44]:
fao_all.set_index('Year').groupby(['Year','Item']).sum()

Value
Year Item                       
2008 Bovine Meat         12444.0
     Meat, Other           283.0
     Mutton & Goat Meat    163.0
     Pigmeat              8928.0
     Poultry Meat        15844.0
2009 Bovine Meat         12258.0
     Meat, Other           281.0
     Mutton & Goat Meat    153.0
     Pigmeat              9005.0
     Poultry Meat        15098.0
2010 Bovine Meat         12072.0
     Meat, Other           279.0
     Mutton & Goat Meat    148.0
     Pigmeat              8702.0
     Poultry Meat        15906.0
2011 Bovine Meat         11592.0
     Meat, Other           280.0
     Mutton & Goat Meat    142.0
     Pigmeat              8396.0
     Poultry Meat        15639.0
2012 Bovine Meat         11725.0
     Meat, Other           286.0
     Mutton & Goat Meat    146.0
     Pigmeat              8552.0
     Poultry Meat        15722.0
2013 Bovine Meat         11587.0
     Meat, Other           290.0
     Mutton & Goat Meat    154.0
     Pigmeat              8729.0
     Poultry Meat        16006.0
2014 Bovine Meat         11889.0
     Meat, Other           287.0
     Mutton & Goat Meat    176.0
     Pigmeat              8137.0
     Poultry Meat        16298.0
2015 Bovine Meat         11275.0
     Meat, Other           291.0
     Mutton & Goat Meat    180.0
     Pigmeat              8366.0
     Poultry Meat        17544.0
2016 Bovine Meat         11630.0
     Meat, Other           292.0
     Mutton & Goat Meat    180.0
     Pigmeat              8489.0
     Poultry Meat        17944.0
2017 Bovine Meat         12039.0
     Meat, Other           290.0
     Mutton & Goat Meat    195.0
     Pigmeat              8543.0
     Poultry Meat        18139.0
2018 Bovine Meat         12155.0
     Meat, Other           291.0
     Mutton & Goat Meat    202.0
     Pigmeat              8999.0
     Poultry Meat        18525.0